## Projeto Final - Redes Neurais

Ilton Albuquerque Martins de Lima - iaml@cesar.school

# Treinamento com interface de alto nível

## Importação das bibliotecas

In [1]:
# http://pytorch.org/
from os.path import exists

import torch

In [2]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from PIL import UnidentifiedImageError
from torch.utils.data import Dataset, random_split

# Dataset

In [3]:
#Importação do Dataset
!curl -L -o /content/archive.zip https://www.kaggle.com/api/v1/datasets/download/sartajbhuvaji/brain-tumor-classification-mri

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 86.7M  100 86.7M    0     0  55.3M      0  0:00:01  0:00:01 --:--:--  128M


In [4]:
#  Criar pasta do dataset
!unzip -q -o /content/archive.zip -d /content/brain-tumor-classification-mri

In [5]:
# Selecionando Pasta do Dataset
brain_tumor_folder = "/content/brain-tumor-classification-mri"

In [6]:
# Processo de Criação do Dataset
transform=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
    ])

In [7]:
# Função para não trazer erros das imagens
class CustomDataset(datasets.ImageFolder):
  def __getitem__(self, index):
    try:
      return super(CustomDataset, self).__getitem__(index)
    except UnidentifiedImageError:
      print(f"Skipped problematic image {index}")
      return None

In [8]:
# Criando Dataset
dataset = CustomDataset(brain_tumor_folder, transform)

In [9]:
# Dividindo Dataset Treino e Teste
dataset_train, dataset_test = random_split(dataset, [0.7, 0.3])

In [10]:
# Tamanho do Dataset Treino
len(dataset_train)

2285

In [11]:
# Tamanho do Dataset Test
len(dataset_test)

979

## DataLoader

In [12]:
# Função para lidar com dados com erro
def custom_collate_fn(batch):
  batch = list(filter(lambda x: x is not None, batch))
  return torch.utils.data.default_collate(batch)

In [24]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(1111)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 100}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

train_loader = torch.utils.data.DataLoader(dataset_train,**train_kwargs, collate_fn=custom_collate_fn)
test_loader = torch.utils.data.DataLoader(dataset_test, **test_kwargs, collate_fn=custom_collate_fn)

## Criação da rede

In [14]:
# Criação da Rede Neural com 64*64 de entrada e 3 de saida
input_size = 64*64
output_size = 4
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 2048),
            nn.ReLU(),
            nn.Linear(2048, 4096),
            nn.ReLU(),
            nn.Linear(4096, 8192),
            nn.ReLU(),
            nn.Linear(8192, 8192),
            nn.ReLU(),
            nn.Linear(8192, 4096),
            nn.ReLU(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 4)
        )

    def forward(self, x):
        x = x.view(-1, input_size)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net()

In [15]:
# Modelo
model

Net(
  (fc): Sequential(
    (0): Linear(in_features=4096, out_features=2048, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2048, out_features=4096, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4096, out_features=8192, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8192, out_features=8192, bias=True)
    (7): ReLU()
    (8): Linear(in_features=8192, out_features=4096, bias=True)
    (9): ReLU()
    (10): Linear(in_features=4096, out_features=2048, bias=True)
    (11): ReLU()
    (12): Linear(in_features=2048, out_features=1024, bias=True)
    (13): ReLU()
    (14): Linear(in_features=1024, out_features=512, bias=True)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=256, bias=True)
    (17): ReLU()
    (18): Linear(in_features=256, out_features=4, bias=True)
  )
)

## Predict

## Treinamento

In [16]:
model(dataset_train[5][0])

tensor([[-1.3338, -1.3919, -1.4158, -1.4058]], grad_fn=<LogSoftmaxBackward0>)

### Criando o objeto de treinamento

In [17]:
# Treinamento
def train(log_interval, dry_run, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [18]:
# Teste
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100. * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        acc))
    return acc

## Avaliação

In [25]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
scheduler = StepLR(optimizer, step_size=4, gamma=0.7)
best_acc = test(model, device, test_loader)
for epoch in range(1, epochs + 1):
    train(10, False, model, device, train_loader, optimizer, epoch)
    acc = test(model, device, test_loader)
    if acc > best_acc:
      torch.save(model.state_dict(), "brain_tumor_nn.pt")
    scheduler.step()


Test set: Average loss: 1.3844, Accuracy: 127/979 (13%)

Train Epoch: 1 [0/2285 (0%)]	Loss: 1.383732
Train Epoch: 1 [640/2285 (28%)]	Loss: 0.439662
Train Epoch: 1 [1280/2285 (56%)]	Loss: 0.471639
Train Epoch: 1 [1920/2285 (83%)]	Loss: 0.253452

Test set: Average loss: 0.5602, Accuracy: 852/979 (87%)

Train Epoch: 2 [0/2285 (0%)]	Loss: 0.567836
Train Epoch: 2 [640/2285 (28%)]	Loss: 0.350952
Train Epoch: 2 [1280/2285 (56%)]	Loss: 0.286835
Train Epoch: 2 [1920/2285 (83%)]	Loss: 0.291434

Test set: Average loss: 0.3501, Accuracy: 852/979 (87%)

Train Epoch: 3 [0/2285 (0%)]	Loss: 0.332062
Train Epoch: 3 [640/2285 (28%)]	Loss: 0.357188
Train Epoch: 3 [1280/2285 (56%)]	Loss: 0.289600
Train Epoch: 3 [1920/2285 (83%)]	Loss: 0.249953

Test set: Average loss: 0.3329, Accuracy: 852/979 (87%)

Train Epoch: 4 [0/2285 (0%)]	Loss: 0.300694
Train Epoch: 4 [640/2285 (28%)]	Loss: 0.388613
Train Epoch: 4 [1280/2285 (56%)]	Loss: 0.268674
Train Epoch: 4 [1920/2285 (83%)]	Loss: 0.275735

Test set: Average l

In [ ]:
aux = torch.load("brain_tumor_nn.pt")

In [ ]:
aux

In [ ]:
model.load_state_dict(aux)

Meu dataset escolhido foi o brain tumor onde as imagens são classificação de tumores cerebrais. Foi usado o tamanho 64*64 e a saída de 4. Usei uma rede um pouco complexa, por se tratar de imagens. Usei o batch_size de treinamento de 64 e teste de 100. E foi usada 10 épocas. A Acurácia deu um máximo de: 87% o que é um bom número e a menor perda foi de: 0.171556 na 10 época.  